<a href="https://colab.research.google.com/github/Omarleel/ParallelMangaTranslator/blob/main/ParallelMangaTranslator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <font color='#018ada'>[1] Clonar repositorio e instalar dependencias</font> { display-mode: "form" }
%cd /content
# Clonar el repositorio
!git clone https://github.com/Omarleel/ParallelMangaTranslator.git
# Accede al proyecto
%cd ParallelMangaTranslator
# Instalar dependencias
!pip install -r requirements.txt

In [ ]:

#@title <font color='#018ada'>[2] Parámetros de entrada</font> { display-mode: "form" }

import os
from loguru import logger
import sys
import warnings
import torch

from Applications.ParallelProcessor import ParallelProcessor
from Applications.ImageProcessor import ImageProcessor
from Applications.Utilities import Utilities
from Utils.Constantes import RUTA_REMOTA
from Applications.GoogleDriveManager import GoogleDriveManager

logger.remove()
warnings.filterwarnings("ignore", message="The class ViTFeatureExtractor is deprecated", category=FutureWarning)
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

if __name__ == '__main__':
    torch.cuda.empty_cache()
    utilities = Utilities()
    utilities.download_fonts()
    utilities.download_models()
    drive_manager = GoogleDriveManager()
    drive_manager.create_folder(RUTA_REMOTA)
    modelo_inpaint = "lama_mpe" #@param ['opencv-tela', 'lama_mpe', 'lama_large_512px', 'aot']
    batch_size = 8 # @param {type:"integer"}
    idioma_entrada = "Japonés" #@param ['Chino', 'Coreano', 'Inglés', 'Japonés']
    idioma_salida = "Español" #@param ['Español', 'Inglés', 'Portugués', 'Francés', 'Italiano', 'Francés']
    url_imagenes = "https://drive.google.com/file/d/..." #@param {type:"string"}
    ruta_carpeta_entrada = utilities.descargar_y_extraer_zip(drive_manager, url_imagenes)
    if ruta_carpeta_entrada is None:
        _, _, ruta_carpeta_entrada = drive_manager.download_folder_by_link(url_imagenes)
    if ruta_carpeta_entrada is None:
        sys.exit()

    ruta_carpeta_salida = os.path.join(ruta_carpeta_entrada, "Outputs")
    ruta_carpeta_limpieza = os.path.join(ruta_carpeta_salida, "Limpieza")
    ruta_carpeta_traduccion = os.path.join(ruta_carpeta_salida, "Traduccion")
    os.makedirs(ruta_carpeta_salida, exist_ok=True)
    os.makedirs(ruta_carpeta_limpieza, exist_ok=True)
    os.makedirs(ruta_carpeta_traduccion, exist_ok=True)

    image_procesor = ImageProcessor(
        idioma_entrada=idioma_entrada,
        idioma_salida=idioma_salida,
        modelo_inpaint=modelo_inpaint
    )

    parallel_processor = ParallelProcessor()
    parallel_processor.procesar_en_paralelo(
        ruta_carpeta_entrada=ruta_carpeta_entrada,
        ruta_carpeta_salida=ruta_carpeta_salida,
        process_func=image_procesor.procesar,
        batch_size=batch_size,
    )

    drive_manager.upload_folder(
        input_folder_path=ruta_carpeta_salida,
        output_folder_name=os.path.splitext(os.path.basename(ruta_carpeta_entrada))[0]
    )
